In [ ]:
!pip install azure-ai-textanalytics==5.2.0b1
!pip install bs4
!pip install requests

In [6]:
# authenticate azure text analytics client
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

key = "677e7a68d2f94932a97572bfd9505ea3"
endpoint = "https://covid-corruption-nlp2.cognitiveservices.azure.com/"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [ ]:
# perform key phrase extraction
def key_phrase_extraction(client, documents=""):
    try:
        responses = client.extract_key_phrases(documents = documents)
        
        key_phrases = []
        for response in responses:
            if response.key_phrases:
                key_phrases += response.key_phrases
        
        return key_phrases

    except Exception as err:
        print("Encountered exception. {}".format(err))


news_source = "sky"
f_in = open('datasets/parsed-news/{}.txt'.format(news_source), 'r')
documents = [line.rstrip() for line in f_in.readlines()]
num_docs = len(documents)

key_phrases = []
for i in range(0, num_docs, 10):
    print(i)
    key_phrase = key_phrase_extraction(
                    client, 
                    documents=documents[i: min(num_docs, i+10)]
    )

    if key_phrase:
        key_phrases += key_phrase

print(key_phrases)
f_out = open('datasets/parsed-news/{}_phrases.txt'.format(news_source), 'w+')
for phrase in key_phrases:
    phrase = '-'.join(phrase.split())
    f_out.write(phrase + '\n')